In [55]:
import json
import itertools
from pyspark.mllib.fpm import FPGrowth
from matplotlib import pyplot as plt
import csv

In [2]:
hubs_tags = sc.textFile('hdfs://master:54310/exp_f/hubs_tags').map(json.loads)

In [4]:
h_t_cartesian = hubs_tags.flatMap(lambda ht: list(itertools.product(ht['Hubs'], ht['Tags'])))

In [ ]:
h_t_cartesian.count()

In [69]:
htc = h_t_cartesian.map(lambda ht: (ht, 1)).reduceByKey(int.__add__).filter(lambda x: x[1] > 14)
sorted_htc = htc.sortBy(lambda htc: htc[1], ascending=False)

In [ ]:
sorted_htc.filter(lambda htc: htc[1]>50).collect()

In [37]:
groupped_by_tag = htc.groupBy(lambda htc: htc[0][1])

In [38]:
t = groupped_by_tag.map(lambda g: len(g[1])).collect()

In [39]:
plt.hist(t, bins=80)
plt.show()

In [50]:
def proceed_g(g):
    tag = g[0]
    htcs = list(g[1])
    

In [51]:
res = groupped_by_tag.map(proceed_g)

In [52]:
res.count()

859

In [70]:
thc = htc.map(lambda x: (x[0][1], x[0][0], x[1]))

In [71]:
res = thc.sortBy(lambda x: (x[0],-x[2])).collect()

In [72]:
with open('/home/hadoop/tag_hub_large.csv', 'w+') as fp:
    writer = csv.writer(fp)
    writer.writerows(res)